In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

In [ ]:
# macOS용 한글 폰트 경로
font_path = "/System/Library/Fonts/Supplemental/AppleGothic.ttf"
font_prop = fm.FontProperties(fname=font_path)
plt.rcParams['font.family'] = font_prop.get_name()
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
import streamlit as st
import pandas as pd
import seaborn as sns
import joblib
import re
from collections import Counter
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
from datetime import datetime
nltk.download('stopwords')

In [ ]:
# 사이드바로 모드 선택
mode = st.sidebar.selectbox("🧭 모드 선택", ["뉴스 판별기", "데이터 분석"])

In [ ]:
# =======================
# 뉴스 판별기
# =======================
if mode == "뉴스 판별기":
    st.markdown("## 🧪 뉴스 기사 가짜/진짜 판별기")
    st.write("뉴스 제목과 내용을 입력하면 AI가 판단해줘요!")

In [ ]:
    @st.cache_resource
    def load_model():
        return joblib.load('fake_news_model.pkl')

In [ ]:
    model = load_model()

In [ ]:
    title = st.text_input("📰 뉴스 제목을 입력하세요")
    content = st.text_area("📄 뉴스 본문을 입력하세요")

In [ ]:
    if st.button("✅ 판별하기"):
        if not content.strip():
            st.warning("❗ 뉴스 본문을 입력해주세요.")
        else:
            full_text = title + " " + content
            result = model.predict_proba([full_text])[0][1]
            if result > 0.5:
                st.error(f"❌ 가짜 뉴스일 확률: {result*100:.1f}%")
            else:
                st.success(f"✅ 진짜 뉴스일 확률: {(1-result)*100:.1f}%")

In [ ]:
    # 단어 빈도 시각화 (불용어 제거)
    st.markdown("---")
    st.markdown("### 📊 가짜 뉴스에서 많이 나온 단어 (Top 20, 불용어 제거)")

In [ ]:
    @st.cache_data
    def get_filtered_top_words():
        try:
            df = pd.read_csv('Fake.csv')
            text = ' '.join(df['text'].dropna().tolist())
            words = re.findall(r'\b\w+\b', text.lower())
            filtered = [w for w in words if w not in stopwords.words('english') and len(w) > 2]
            counter = Counter(filtered)
            return counter.most_common(20)
        except:
            return []

In [ ]:
    top_words = get_filtered_top_words()
    st.table(pd.DataFrame(top_words, columns=["단어", "횟수"]))

In [ ]:
    if top_words:
        words, counts = zip(*top_words)
        fig, ax = plt.subplots(figsize=(8, 10))
        ax.barh(words[::-1], counts[::-1], color='tomato')
        ax.set_title('가짜 뉴스에서 가장 많이 나온 단어')
        ax.set_xlabel('등장 횟수')
        ax.set_ylabel('단어')
        st.pyplot(fig)
    else:
        st.info("⚠️ 표시할 단어가 없습니다.")

In [ ]:
# =======================
# 데이터 분석
# =======================
elif mode == "데이터 분석":
    st.markdown("## 📊 가짜 뉴스 데이터 분석")

In [ ]:
    @st.cache_data
    def load_data():
        df = pd.read_csv("merged_data_complete.csv")
        df["sentiment_score"] = pd.to_numeric(df["sentiment_score"], errors="coerce")
        df["date"] = pd.to_datetime(df["date"], errors="coerce")
        if "sentiment_label" not in df.columns or df["sentiment_label"].isnull().sum() > 0:
            def get_sentiment_label(score):
                if pd.isna(score): return None
                if score > 0.2:
                    return "긍정"
                elif score < -0.2:
                    return "부정"
                else:
                    return "중립"
            df["sentiment_label"] = df["sentiment_score"].apply(get_sentiment_label)
        return df

In [ ]:
    df = load_data()

In [ ]:
    analysis_option = st.selectbox("분석 항목을 선택하세요", [
        "제목 단어 수 분포",
        "본문 단어 수 분포",
        "감정 점수 분포",
        "감정 결과 분포",
        "자극 단어 포함 비율",
        "시기별 FAKE 뉴스 트렌드",
        "카테고리별 FAKE 뉴스 비율",
        "FAKE/REAL 워드클라우드"
    ])

In [ ]:
    if analysis_option == "제목 단어 수 분포":
        st.subheader("✍️ 제목 단어 수 분포")
        fig, ax = plt.subplots()
        sns.boxplot(data=df, x="label_text", y="title_word_count", ax=ax)
        ax.set_title("FAKE vs REAL - 제목 단어 수 분포 (Boxplot)")
        st.pyplot(fig)

In [ ]:
    elif analysis_option == "본문 단어 수 분포":
        st.subheader("📄 본문 단어 수 분포")
        fig, ax = plt.subplots()
        sns.violinplot(data=df, x="label_text", y="text_word_count", ax=ax)
        ax.set_title("FAKE vs REAL - 본문 단어 수 분포 (Violin Plot)")
        st.pyplot(fig)

In [ ]:
    elif analysis_option == "감정 점수 분포":
        st.subheader("📉 감정 점수 분포")
        fig, ax = plt.subplots()
        sns.kdeplot(data=df, x="sentiment_score", hue="label_text", ax=ax, fill=True)
        ax.set_title("감정 점수 분포 (FAKE vs REAL)")
        st.pyplot(fig)

In [ ]:
    elif analysis_option == "감정 결과 분포":
        st.subheader("😐 감정 결과 분포")
        fig, ax = plt.subplots()
        sns.countplot(data=df, x="sentiment_label", hue="label_text", ax=ax)
        ax.set_title("감정 레이블 분포 (긍정/부정/중립)")
        st.pyplot(fig)

In [ ]:
    elif analysis_option == "자극 단어 포함 비율":
        st.subheader("🚨 자극적인 단어 포함 비율")
        buzzwords = ["shocking", "scandal", "explosive", "truth", "bombshell"]
        def has_buzz(text):
            text = str(text).lower()
            return any(word in text for word in buzzwords)
        df["buzz"] = df["text"].apply(has_buzz)
        buzz_ratio = df.groupby("label_text")["buzz"].mean()
        fig, ax = plt.subplots()
        ax.pie(buzz_ratio, labels=buzz_ratio.index, autopct='%1.1f%%', startangle=140, colors=['#ff9999','#66b3ff'])
        ax.set_title("FAKE vs REAL - 자극 단어 포함 비율")
        st.pyplot(fig)

In [ ]:
    elif analysis_option == "시기별 FAKE 뉴스 트렌드":
        st.subheader("📅 시기별 FAKE 뉴스 생성 추이")
        df_f = df[df["label_text"] == "FAKE"].dropna(subset=["date"])
        df_f["month"] = df_f["date"].dt.to_period("M").astype(str)
        trend = df_f["month"].value_counts().sort_index()
        fig, ax = plt.subplots(figsize=(10, 4))
        sns.lineplot(x=trend.index, y=trend.values, marker="o", ax=ax)
        ax.set_title("월별 FAKE 뉴스 생성량 추이")
        ax.set_xlabel("월")
        ax.set_ylabel("뉴스 수")
        plt.xticks(rotation=45)
        st.pyplot(fig)

In [ ]:
    elif analysis_option == "카테고리별 FAKE 뉴스 비율":
        st.subheader("🗂 카테고리별 FAKE 뉴스 비율")
        if "subject" in df.columns:
            ratio = df.groupby("subject")["label"].mean().sort_values()
            fig, ax = plt.subplots(figsize=(8, 6))
            sns.barplot(x=ratio.values, y=ratio.index, ax=ax, palette="coolwarm")
            ax.set_title("카테고리별 FAKE 뉴스 비율")
            ax.set_xlabel("FAKE 비율")
            st.pyplot(fig)
        else:
            st.warning("⚠️ subject 컬럼이 없습니다.")

In [ ]:
    elif analysis_option == "FAKE/REAL 워드클라우드":
        st.subheader("☁️ FAKE vs REAL 워드클라우드")
        stop_words = set(stopwords.words('english'))
        fake_text = ' '.join(df[df['label_text'] == "FAKE"]["text"].dropna())
        real_text = ' '.join(df[df['label_text'] == "REAL"]["text"].dropna())

In [ ]:
        fake_wc = WordCloud(width=600, height=400, background_color='white', stopwords=stop_words).generate(fake_text)
        real_wc = WordCloud(width=600, height=400, background_color='white', stopwords=stop_words).generate(real_text)

In [ ]:
        col1, col2 = st.columns(2)
        with col1:
            st.markdown("**FAKE 뉴스**")
            st.image(fake_wc.to_array())
        with col2:
            st.markdown("**REAL 뉴스**")
            st.image(real_wc.to_array())